In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [2]:
df = pd.read_csv("BL Wowcher Wowcher Revenue BL CSS Spend Data_2camps.csv")
df.head()

,Day,Hour of the day,Campaign status,Campaign,Clicks,Impr.,CTR,Avg. CPC,Cost,In-Visit Net Revenue,In-Visit Net ROI,New customers,Bid strategy,Conversions,Conv. value,All conv.,All conv. value,Conv. rate,Conv. value / cost,Cost / conv.
0,16-12-2020,0,Enabled,BL|Smart Shopping|Cameras & Optics,17,928,1.83%,0.29,4.98,£0.00,0.0,0,--,1,0,50,49,5.88%,0.0,4.98
1,16-12-2020,1,Enabled,BL|Smart Shopping|Cameras & Optics,6,395,1.52%,0.23,1.36,£0.00,0.0,0,--,0,0,9,9,0.00%,0.0,0.00
2,16-12-2020,2,Enabled,BL|Smart Shopping|Cameras & Optics,6,201,2.99%,0.34,2.02,£0.00,0.0,0,--,1,0,15,14,16.67%,0.0,2.02
3,16-12-2020,3,Enabled,BL|Smart Shopping|Cameras & Optics,6,237,2.53%,0.31,1.85,£0.00,0.0,0,--,0,0,31,31,0.00%,0.0,0.00
4,16-12-2020,4,Enabled,BL|Smart Shopping|Cameras & Optics,5,211,2.37%,0.24,1.18,£0.00,0.0,0,--,0,0,11,11,0.00%,0.0,0.00


In [3]:

# Convert the 'Day' column to datetime
df['Day'] = pd.to_datetime(df['Day'], format='%d-%m-%Y')
df.sort_values(by='Day', inplace=True)

soft_warning_threshold = 1.5
hard_warning_threshold = 2.0

# Initialize soft and hard warning columns as integers (0 and 1)
df['SoftWarning'] = 0
df['HardWarning'] = 0

# Create a new column to store the daily averages for each campaign and day
df['daily_average'] = df.groupby(['Campaign', 'Day'])['Cost'].transform('mean')

# Create a new column 'ShiftedDailyAverage' that shifts the 'daily_average' column 24 times for each campaign
df['ShiftedDailyAverage'] = df.groupby('Campaign')['daily_average'].transform(lambda x: x.shift(24))

for campaign in df['Campaign'].unique():
    campaign_mask = df['Campaign'] == campaign
    campaign_data = df[campaign_mask]
    
    # Calculate the soft and hard warnings based on the specified thresholds
    soft_warning_mask = campaign_data['Cost'] > soft_warning_threshold * campaign_data['ShiftedDailyAverage']
    hard_warning_mask = campaign_data['Cost'] > hard_warning_threshold * campaign_data['ShiftedDailyAverage']
    
    # Update the main DataFrame with the calculated warnings
    df.loc[campaign_mask, 'SoftWarning'] = soft_warning_mask.astype(int)
    df.loc[campaign_mask, 'HardWarning'] = hard_warning_mask.astype(int)

In [4]:
df.head()

,Day,Hour of the day,Campaign status,Campaign,Clicks,Impr.,CTR,Avg. CPC,Cost,In-Visit Net Revenue,...,Conv. value,All conv.,All conv. value,Conv. rate,Conv. value / cost,Cost / conv.,SoftWarning,HardWarning,daily_average,ShiftedDailyAverage
0,2020-12-16,0,Enabled,BL|Smart Shopping|Cameras & Optics,17,928,1.83%,0.29,4.98,£0.00,...,0,50,49,5.88%,0.0,4.98,0,0,15.350417,NaN
26,2020-12-16,2,Enabled,BL|Smart Shopping|Clothing & Accessories,40,"2,507",1.60%,0.28,11.06,£0.00,...,0,101,99,5.00%,0.0,5.53,0,0,62.563333,NaN
27,2020-12-16,3,Enabled,BL|Smart Shopping|Clothing & Accessories,31,"1,711",1.81%,0.33,10.15,£0.00,...,0,56,56,0.00%,0.0,0.00,0,0,62.563333,NaN
28,2020-12-16,4,Enabled,BL|Smart Shopping|Clothing & Accessories,16,"1,311",1.22%,0.25,3.92,£0.00,...,0,40,38,12.50%,0.0,1.96,0,0,62.563333,NaN
29,2020-12-16,5,Enabled,BL|Smart Shopping|Clothing & Accessories,29,"1,654",1.75%,0.26,7.64,£0.00,...,0,49,47,6.90%,0.0,3.82,0,0,62.563333,NaN


In [6]:
df.to_csv("usecase1.csv", index=False)

In [7]:
df.value_counts("SoftWarning")

SoftWarning
0    13230
1     4290
dtype: int64

In [8]:
df.value_counts("HardWarning")

HardWarning
0    15703
1     1817
dtype: int64